In [1]:
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

## Step 1: Data Cleaning

### Choose year range from 2006-2020

In [2]:
YEAR_MIN = 2006
YEAR_MAX = 2020

### Choose one of the targets that you worked in Phase 1

**Target: 4.1** By 2030, ensure that all girls and boys complete free, equitable and quality primary and secondary education leading to relevant and effective learning outcomes

**Related indicator**: Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)

In [3]:
INDICATOR = 'Educational attainment, at least completed upper secondary, population 25+, total (%) (cumulative)'

In [4]:
df = pd.read_excel('Target4.4.xlsx', engine='openpyxl').dropna(how='all')
df = df[df['Series Name'].eq(INDICATOR)]
print(df.shape)
df.head()

(266, 22)


,Series Name,Series Code,Country Name,Country Code,2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],2008 [YR2008],2009 [YR2009],...,2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021]
532,"Educational attainment, at least completed upp...",SE.SEC.CUAT.UP.ZS,Afghanistan,AFG,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.46131
533,"Educational attainment, at least completed upp...",SE.SEC.CUAT.UP.ZS,Albania,ALB,NaN,NaN,NaN,NaN,39.889320,NaN,...,45.279148,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
534,"Educational attainment, at least completed upp...",SE.SEC.CUAT.UP.ZS,Algeria,DZA,NaN,NaN,24.152861,NaN,25.416519,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,"Educational attainment, at least completed upp...",SE.SEC.CUAT.UP.ZS,American Samoa,ASM,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,"Educational attainment, at least completed upp...",SE.SEC.CUAT.UP.ZS,Andorra,AND,51.454182,43.910172,49.950649,NaN,NaN,NaN,...,NaN,NaN,47.83741,47.5452,47.349689,NaN,NaN,NaN,NaN,NaN


### Remove the data of  countries which have 3 or less values for chosen indicator


In [5]:
df = df.set_index('Country Code').filter(like='20')
df.columns = df.columns.str[:4].astype(int)
df = df.loc[:, YEAR_MIN:YEAR_MAX]
df = df[(~df.isna()).sum(axis=1) > 3]
df

,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
Country Code,,,,,,,,,,,,,,,
AND,49.950649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,47.837410,47.545200,47.349689,NaN,NaN,NaN,NaN
ARM,NaN,NaN,NaN,NaN,NaN,90.237610,NaN,NaN,NaN,91.837433,NaN,90.096649,NaN,NaN,89.880859
AUS,66.713837,68.189781,70.088959,70.855743,72.798973,74.061668,76.483017,72.037354,73.595963,75.547577,76.418037,77.261360,78.325760,79.047928,80.053032
AUT,NaN,73.590607,72.469460,75.810188,76.556099,76.427696,76.975113,77.306358,78.426590,79.012901,79.368980,79.662582,NaN,80.674049,NaN
AZE,NaN,88.435417,88.540138,88.610489,88.645401,88.658783,88.713089,88.719742,88.714172,88.665077,88.664452,88.501228,88.430939,88.419579,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
USA,85.723228,NaN,86.614128,86.709084,87.149406,87.590889,87.679100,88.150307,88.313583,88.411972,89.092361,NaN,89.805359,NaN,90.940521
URY,24.739000,NaN,26.977221,27.750971,27.173599,27.018459,27.543711,27.578899,28.453150,28.747761,28.747801,30.018440,30.441191,31.510580,NaN
UZB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91.540611,91.947632,92.079582,92.564621,NaN,96.149246,96.676292,NaN


### Impute your data using all of the following methods.
Each of the method will give you an imputed dataset (i.e. you need to save the imputed data into a file). For the next step, you will work with imputed datasets obtain by applying the following methods. 
#### Constant value
* Mean
* Median
#### Time-series based
* Forward fill
* Backward fill
#### Advanced Method
* Linear regression
* Linear interpolation (extra)

In [8]:
df_mean = df.T.fillna(df.mean(axis=1)).T
df_median = df.T.fillna(df.median(axis=1)).T
df_ffil = df.fillna(method='ffill', axis='columns').fillna(method='bfill', axis='columns') # do bfill to fill empty values in the beginning
df_bfil = df.fillna(method='bfill', axis='columns').fillna(method='ffill', axis='columns') # do ffill to fill empty values in the end
df_regr = pd.DataFrame(
    IterativeImputer(imputation_order='roman').fit_transform(df),
    index=df.index, columns=df.columns
)

# Additinally, data imputation by linear interploation was performed
df_lint = df.interpolate(method='linear', axis='columns').fillna(method='bfill', axis='columns') # do bfill to fill empty values in the beginning


### Each of these dataframes need to be saved in multiple files 

In [9]:
df_mean.to_csv('t4_upsec_mean.csv')
df_median.to_csv('t4_upsec_median.csv')
df_ffil.to_csv('t4_upsec_ffil.csv')
df_bfil.to_csv('t4_upsec_bfil.csv')
df_lint.to_csv('t4_upsec_lint.csv')
df_regr.to_csv('t4_upsec_regr.csv')